<center><img src="https://raw.githubusercontent.com/wangyu16/PolymerScienceEducation/master/Fig/Logo_black.svg" width='150' /></center>

# Network Formation

-----------

## What does this simulator do and how to use it?

This simulator shows the process of the polymerization of AA, A3 and BB type monomers. The total number of A functional groups is equal or less than B type functional groups.

Users can select the total number of monomers, the ratio between A3 and AA type monomers, and the ratio between A and B type functional groups. Simply run all cells, the overall number average degree of polymerization, the sol-gel fractions, and the information of the sol fraction will be shown below.  

In [248]:
#@title 1. Install and import packages {display-mode: "form" }
%%capture
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import random
import math
from IPython.display import clear_output
!pip install sigfig
from sigfig import round
from collections import Counter

In [249]:
#@title 2. Simulation conditions {display-mode: "form"}
#@markdown Set the total number of monomers as $10^n$.
n = 5 #@param {type:"slider", min:4, max:6, step:0.1}
nTotal = math.ceil(10**n)
#@markdown The number of A3 type monomers vs that of AA type monomers.
A3toAA = 0.3 #@param {type:"slider", min:0.1, max:2, step:0.1}
#@markdown The ratio between A and B types of functional groups.
AtoB = 0.95 #@param {type:"slider", min:0.8, max: 1, step: 0.01}

nAA = math.ceil(nTotal/(1+A3toAA+(2+3*A3toAA)/(2*AtoB)))
nA3 = math.ceil(nAA*A3toAA)
nBB = nTotal - nA3 - nAA
nA0 = 3*nA3 + 2*nAA
nB0 = 2*nBB
fCaro = 2*nA0/nTotal

r = AtoB
rho = 3*nA3/nA0
#print(rho)

pcCaro = 2/fCaro
pcStat = 1/math.sqrt(r+r*rho)

if pcCaro < 0.95:
  pTarget = pcCaro + 0.04
else:
  pTarget = 0.99

print(f"The total number of monomers is {nTotal}.")
print(f"The number of AA type monomers is {nAA}, that of A3 is {nA3}, and that of BB is {nBB}.")
print(f"The gelation point is {round(pcCaro, sigfigs=3)} predicted by Carothers theory")
print(f"The gelation point is {round(pcStat, sigfigs=3)} predicted by statistical approach.")

The total number of monomers is 100000.
The number of AA type monomers is 35382, that of A3 is 10615, and that of BB is 54003.
The gelation point is 0.975 predicted by Carothers theory
The gelation point is 0.896 predicted by statistical approach.


In [250]:
%%time
#@title 3. Simulation and Results {display-mode: "form"}
# Create an empty graph
G = nx.Graph()

# Add nodes with types
for i in range(nAA):
    G.add_node(f"AA{i}", type="AA", functionality=2, color='red')
for i in range(nA3):
    G.add_node(f"A3{i}", type="A3", functionality=3, color='purple')
for i in range(nBB):
    G.add_node(f"BB{i}", type="BB", functionality=2, color='blue')


CPU times: user 178 ms, sys: 2 ms, total: 180 ms
Wall time: 180 ms


In [251]:
#@title {display-mode: "form"}

def choose_node_by_type(graph, node_type, n):
  """
  Choose a random node from the graph based on its type and functionality.
  Use the value of functionality as the weight for the random choice.
  Args:
    graph: NetworkX graph
    node_type: List of node types to choose from.
  Returns:
    Random node from the graph.
  """
  # Get nodes of the specified type and their functionalities
  nodes = [(n, d['functionality']) for n, d in graph.nodes(data=True) if d['type'] in node_type]

  # Create a weighted list based on functionality
  weighted_nodes = []
  for node, functionality in nodes:
    if functionality > 0:
      weighted_nodes.extend([node] * functionality)

  # Choose n random nodes from the weighted list
  chosen_nodes = random.sample(weighted_nodes, min(n, len(weighted_nodes)))
  return chosen_nodes

def add_edges_pairwise(graph, listA, listB):
  """Adds edges pairwise between nodes in two lists."""
  for nodeA, nodeB in zip(listA, listB):
    graph.add_edge(nodeA, nodeB)
    graph.nodes[nodeA]['functionality'] -= 1
    graph.nodes[nodeB]['functionality'] -= 1


def sum_functionalities_by_type(graph, node_type):
  """
  Calculates the sum of functionalities of all nodes of given types.
  Args:
    graph: NetworkX graph
    node_type: List of node types to calculate functionalities for.
  Returns:
    Sum of functionalities of nodes of the specified types.
  """
  total_functionality = sum(d['functionality'] for n, d in graph.nodes(data=True) if d['type'] in node_type)
  return total_functionality


In [252]:
# @title {display-mode:"form"}
%%time
DPs = np.array([len(c) for c in nx.connected_components(G)])
result = pd.DataFrame({'pA':[0.0], 'pB':[0.0], 'DPs': [DPs], 'largest':[1/nTotal]})
pA = 0
pB = 0
nA = sum_functionalities_by_type(G, ['AA','A3'])
nB = sum_functionalities_by_type(G, ['BB'])
i=1

while pTarget > pA and pTarget > pB:
  stepSize = math.ceil(nA/100)
  chosen_node_A = choose_node_by_type(G, ['AA','A3'], stepSize)

  chosen_node_B = choose_node_by_type(G, ['BB'], stepSize)
  if not chosen_node_B:
    print(f"No B type nodes with functionality greater than 0.")
    break

  add_edges_pairwise(G, chosen_node_A, chosen_node_B)
  DPs = np.sort(np.array([len(c) for c in nx.connected_components(G)]))
  largest = DPs[-1]
  nA = sum_functionalities_by_type(G, ['AA','A3'])
  nB = sum_functionalities_by_type(G, ['BB'])
  pA = 1-nA/nA0
  pB = 1-nB/nB0
  result = pd.concat([result, pd.DataFrame({'pA':[pA], 'pB':[pB], 'DPs': [DPs], 'largest':largest/nTotal})], ignore_index=True)
  print(f"Step {i}: The reaction extent of A is {round(pA, sigfigs=3)}, the step size is {len(chosen_node_A)}.")
  i+=1

clear_output()

CPU times: user 2min 11s, sys: 665 ms, total: 2min 11s
Wall time: 2min 22s


In [253]:
#@title Show results {display-mode:"form"}
nMolecules = nx.number_connected_components(G)
solFraction = 1-result['largest'].iloc[-1]
print(f"The number of molecules at reaction extent {round(result['pA'].iloc[-1], sigfigs=3)} is {nMolecules}.")
print(f"The number average degree of polymerization is {round(nTotal/nMolecules, sigfigs=3)}.")
print(f"The sol fraction is {round(solFraction, sigfigs=3)}.")

The number of molecules at reaction extent 0.99 is 772.
The number average degree of polymerization is 130.0.
The sol fraction is 0.0518.


In [254]:
# @title Plot the results {display-mode: "form"}
ylim = 0.5


fig = go.Figure()
fig.add_trace(go.Scatter(mode='lines+markers', name='Largest', x=result['pA'], y=result['largest']))
fig.add_shape(
    type="line",
    x0=pcStat,
    y0=0,
    x1=pcStat,
    y1=1,
    line=dict(color="orange", width=3, dash="dash"),
    xref='x',
    yref='paper'
)

fig.add_shape(
    type="line",
    x0=pcCaro,
    y0=0,
    x1=pcCaro,
    y1=1,
    line=dict(color="red", width=3, dash="dash"),
    xref='x',
    yref='paper'
)

# Set labels and title
fig.update_layout(
    title='Weight fraction of the largest molecule',
    xaxis_title='pA',
    xaxis_range=[pcStat-0.1,pcCaro+0.1],
    yaxis_range=[0,ylim]
)

# Display the plot
fig.show()

In [255]:
# @title {display-mode: "form"}
DPnoLargest = result['DPs'].apply(lambda x: np.mean(x[:-1]))
ylim = DPnoLargest.max()*4

fig = go.Figure()
fig.add_trace(go.Scatter(mode='lines+markers', name='Mean DP', x=result['pA'], y=result['DPs'].apply(np.mean)))
fig.add_trace(go.Scatter(mode='lines+markers', name='Mean DP without the largest', x=result['pA'], y=DPnoLargest))
fig.add_shape(
    type="line",
    x0=pcStat,
    y0=0,
    x1=pcStat,
    y1=1,
    line=dict(color="orange", width=3, dash="dash"),
    xref='x',
    yref='paper'
)

fig.add_shape(
    type="line",
    x0=pcCaro,
    y0=0,
    x1=pcCaro,
    y1=1,
    line=dict(color="red", width=3, dash="dash"),
    xref='x',
    yref='paper'
)

# Set labels and title
fig.update_layout(
    title='Average DP with or without the largest molecule',
    xaxis_title='pA',
    xaxis_range=[pcStat-0.1,pcCaro+0.1],
    yaxis_range=[0,ylim]
)

# Display the plot
fig.show()

In [256]:
# @title {display-mode: "form"}
monomer = result['DPs'].apply(lambda x: x.tolist().count(1))
dimer = result['DPs'].apply(lambda x: x.tolist().count(2))
trimer = result['DPs'].apply(lambda x: x.tolist().count(3))
pentamer = result['DPs'].apply(lambda x: x.tolist().count(5))
decotamer = result['DPs'].apply(lambda x: x.tolist().count(10))
twenticamer = result['DPs'].apply(lambda x: x.tolist().count(20))

fig = go.Figure()

fig.add_trace(go.Scatter(x=result['pA'], y=monomer, mode='lines+markers', name='Monomer'))
fig.add_trace(go.Scatter(x=result['pA'], y=dimer, mode='lines+markers', name='Dimer'))
fig.add_trace(go.Scatter(x=result['pA'], y=trimer, mode='lines+markers', name='Trimer'))
fig.add_trace(go.Scatter(x=result['pA'], y=pentamer, mode='lines+markers', name='Pentamer'))
fig.add_trace(go.Scatter(x=result['pA'], y=decotamer, mode='lines+markers', name='Decotamer'))
fig.add_trace(go.Scatter(x=result['pA'], y=twenticamer, mode='lines+markers', name='Twenticamer'))

# Set labels and title
fig.update_layout(
    xaxis_title='pA',
    yaxis_title='Frequency',
    yaxis_range=[0,dimer.max()*1.1],
    xaxis_range=[0,1]
)

# Display the plot
fig.show()

In [257]:
# @title {display-mode: "form"}
last = result['DPs'].apply(lambda x: x[-1])
second = result['DPs'].apply(lambda x: x[-2])
third = result['DPs'].apply(lambda x: x[-3])
fifth = result['DPs'].apply(lambda x: x[-5])
tenth = result['DPs'].apply(lambda x: x[-10])

# Create a Plotly scatter plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=result['pA'], y=last, mode='lines+markers', name='Largest'))
fig.add_trace(go.Scatter(x=result['pA'], y=second, mode='lines+markers', name='Second largest'))
fig.add_trace(go.Scatter(x=result['pA'], y=third, mode='lines+markers', name='Third largest'))
fig.add_trace(go.Scatter(x=result['pA'], y=fifth, mode='lines+markers', name='Fifth largest'))
fig.add_trace(go.Scatter(x=result['pA'], y=tenth, mode='lines+markers', name='Tenth largest'))

fig.add_shape(
    type="line",
    x0=pcStat,
    y0=0,
    x1=pcStat,
    y1=1,
    line=dict(color="orange", width=3, dash="dash"),
    xref='x',
    yref='paper'
)

fig.add_shape(
    type="line",
    x0=pcCaro,
    y0=0,
    x1=pcCaro,
    y1=1,
    line=dict(color="red", width=3, dash="dash"),
    xref='x',
    yref='paper'
)

# Set labels and title
fig.update_layout(
    title='Largest molecules vs pA',
    xaxis_title='pA',
    yaxis_title='DP',
    xaxis_range=[pcStat-0.1,pcCaro+0.1],
    yaxis_range=[0,second.max()*1.1]
)

# Display the plot
fig.show()

In [258]:
# @title {display-mode: "form"}
def get_subgraph_by_size(graph, desired_size):
  """
  Finds a connected subgraph of a specified size.

  Args:
    graph: A NetworkX graph.
    desired_size: The desired size of the subgraph.

  Returns:
    A NetworkX subgraph of the desired size, or None if none is found.
  """
  for component in nx.connected_components(graph):
    if len(component) == desired_size:
      return graph.subgraph(component)
  return None  # No subgraph of the desired size found

def plotGraph(G):
  # Calculate node positions (you can use different layout algorithms)
  pos = nx.spring_layout(G, dim=3)  # 3D layout

  # Extract node coordinates
  Xn = [pos[k][0] for k in G.nodes()]
  Yn = [pos[k][1] for k in G.nodes()]
  Zn = [pos[k][2] for k in G.nodes()]

  # Create edges
  edge_x = []
  edge_y = []
  edge_z = []
  for edge in G.edges():
      x0, y0, z0 = pos[edge[0]]
      x1, y1, z1 = pos[edge[1]]
      edge_x.extend([x0, x1, None])
      edge_y.extend([y0, y1, None])
      edge_z.extend([z0, z1, None])

  # Map node types to numerical values
  type_mapping = {'AA': 0, 'A3': 1, 'BB': 2}
  node_colors = [type_mapping[d['type']] for n, d in G.nodes(data=True)]

  # Create node and edge traces
  node_trace = go.Scatter3d(x=Xn, y=Yn, z=Zn,
                          mode='markers',
                          hoverinfo='text',
                          marker=dict(
                              showscale=True,  # Enable colorscale
                              colorscale='YlGnBu',
                              reversescale=True,
                              color=node_colors,  # Use numerical color values
                              size=3,
                              colorbar=dict(
                                  thickness=15,
                                  title='Node Type',
                                  xanchor='left',
                                  titleside='right'
                              ),
                              line_width=2))

  edge_trace = go.Scatter3d(x=edge_x, y=edge_y, z=edge_z,
                          mode='lines',
                          line=dict(color='black', width=2))

  # Create figure and layout
  fig = go.Figure(data=[edge_trace, node_trace],
              layout=go.Layout(
                  showlegend=False,
                  scene=dict(
                      xaxis_title='X',
                      yaxis_title='Y',
                      zaxis_title='Z'
                  ),
                  margin=dict(t=50),
                  hovermode='closest',
                  height=800
              ))

  fig.show()

In [259]:
# @title {display-mode: "form"}
# Example usage:
subgraph = get_subgraph_by_size(G, result['DPs'].iloc[-1][-2])  # Replace G with your NetworkX graph and 10 with the desired size

if subgraph:
  print("The second largest network.")
  plotGraph(subgraph)
else:
  print("No subgraph of given size found.")

The second largest network.


In [260]:
#@title Copyright { display-mode: "form" }
import requests
from IPython.display import Markdown
copyright = requests.get("https://raw.githubusercontent.com/wangyu16/PolymerScienceEducation/master/copyright.md")
Markdown(copyright.text)

### MIT License  

Copyright (c) 2021 -- 2024 Yu Wang

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### Contact

Dr. Yu Wang  
Department of Chemistry  
Institute for Materials Research and Innovation  
University of Louisiana at Lafayette  
Lafayette, LA 70504  
Email: yuwang@louisiana.edu

### Acknowledgement

This project is sponsored by National Science Foundation (NSF-2142043). 
